# Resolve the Sidereal Time at Midnight of Site
- Define the site's longitude/latitude and the timezone.
- Figure out the dawn/dusk of the site over a range of days.
- Figure out the sidereal time at the midnight of these days at the site.
- The sidereal time now gives us the reference of the RA window that the telescope at the site may observe.

In [1]:
import requests
import json
import pprint
import pandas as pd
import numpy as np
import requests
import json
import datetime as dt
from datetime import datetime, timedelta, date
from zoneinfo import ZoneInfo
import pytz
from astral import LocationInfo
from astral.sun import sun

In [2]:
def create_midnight_datetime(timezone_str, date_val) -> datetime:
    """
    Conver the date_val to the midnight of the localized timezone. 
    """
    # Create a naive datetime object for midnight of the given date
    if(isinstance(date_val, str)):
        naive_datetime = datetime.strptime(date_val, '%Y-%m-%d').replace(hour=0, minute=0, second=0, microsecond=0)
    else:
        naive_datetime = datetime(date_val.year, date_val.month, date_val.day)
    
    # Convert naive datetime to a timezone-aware datetime using ZoneInfo
    timezone = ZoneInfo(timezone_str)
    localized_datetime = naive_datetime.replace(tzinfo=timezone)
    
    return localized_datetime

def to_hhmmss(localized_datetime:datetime, tz=pytz.utc) -> str:
    """
    Convert the localized_datetime to the corresponding hh:mm:ss of the timezon tz. 
    """
    return localized_datetime.astimezone(tz).strftime('%H:%M:%S')

# MST is -7 hours comparing with UTC
print(to_hhmmss(create_midnight_datetime("MST", date(2024, 2, 1))))
print(to_hhmmss(create_midnight_datetime("MST", "2024-02-01")))

07:00:00
07:00:00


In [10]:
def fetch_sidereal_time(d: str, latitude, longitude, reps, intv_mag=24, intv_unit="hours", time:str="00:00:00"):
    """
    Fetch the sidereal_time given the input.

    :param d: yyyy-mm-dd
    :param longitude: longitude
    :param latitude: latitude
    :param reps: The number of repetitions.
    :param intv_mag: The amount of interval.
    :param intv_unit: The unit of interval.
    :param time: the sun time of the corresponding sidereal time.
    """
    
    # URL of the API endpoint
    # url = "https://aa.usno.navy.mil/api/siderealtime?date=2020-03-04%20&coords=41.89,%2012.48&reps=3%20&intv_mag=12&intv_unit=hours%20&time=21:00:00"
    url = f"https://aa.usno.navy.mil/api/siderealtime?date={d}%20&coords={latitude},%20{longitude}&reps={reps}%20&intv_mag={intv_mag}&intv_unit={intv_unit}%20&time={time}"
    print(url)
    # Make a GET request to fetch the raw JSON data
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON data
        return response.json()
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        raise Error()

In [36]:
days=3

start_date = dt.date(2024, 8, 1)
end_date = start_date + dt.timedelta(days=days-1)

loc_meta = [
    {
        "site":"mt_lemmon",
        "lat": 32.4420,
        "long": -110.789,
        "tz": "MST",
    },
    {
        "site": "holmdel",
        "lat": 40.3541,
        "long":-74.184,
        "tz": "America/New_York",
    }
    ]
loc_meta = pd.DataFrame(loc_meta)

date_list = [(start_date + dt.timedelta(days=x)) for x in range((end_date - start_date).days + 1)]
loc_meta

,site,lat,long,tz
0,mt_lemmon,32.4420,-110.789,MST
1,holmdel,40.3541,-74.184,America/New_York


In [37]:
data = []
for index, row in loc_meta[['site', 'long', 'lat']].iterrows():
    site, long, lat = row
    loc_info = LocationInfo(long, lat)
    data.append(pd.DataFrame([{"site": site, "date":d, **sun(loc_info.observer, date=d)} for d in date_list]))

df_sun_rise_set = pd.concat(data, axis=0)

df = pd.merge(loc_meta, df_sun_rise_set, left_on='site', right_on='site')
df['date'] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [38]:
rows = []
for index, row in df[['site', 'long', 'lat', 'tz']].drop_duplicates().iterrows():
    site, long, lat, tz = row
    d = start_date.strftime("%Y-%m-%d")
    hhmmss = to_hhmmss(create_midnight_datetime(tz, d))
    raw_sidereal = fetch_sidereal_time(d, lat, long, reps=days, intv_mag=1, intv_unit="days", time=hhmmss)
    data_ele = raw_sidereal["properties"]["data"]
    df_sidereal = pd.DataFrame(data_ele)
    df_sidereal['site']=site
    df_sidereal['date'] = pd.to_datetime(df_sidereal[['year', 'month', 'day']])
    df_sidereal['date'] = df_sidereal['date'].dt.strftime('%Y-%m-%d')
    df_sidereal.drop(columns=['year', 'month', 'day'], inplace=True)
    rows.append(df_sidereal)
df_sidereal = pd.concat(rows, axis=0)

https://aa.usno.navy.mil/api/siderealtime?date=2024-08-01%20&coords=32.442,%20-110.789&reps=3%20&intv_mag=1&intv_unit=days%20&time=07:00:00
https://aa.usno.navy.mil/api/siderealtime?date=2024-08-01%20&coords=40.3541,%20-74.184&reps=3%20&intv_mag=1&intv_unit=days%20&time=04:00:00


In [39]:
df_complete=pd.merge(df, df_sidereal, on=['site', 'date'])
df_complete = df_complete[['site', 'date', 'long', 'lat', 'tz', 'dawn', 'sunrise', 
       'sunset', 'dawn', 'dusk', 'noon', 'eqofeq', 'gast', 'gmst', 'last', 'lmst', 'ut1time']]
df_complete

,site,date,long,lat,tz,dawn,sunrise,sunset,dawn,dusk,noon,eqofeq,gast,gmst,last,lmst,ut1time
0,mt_lemmon,2024-08-01,-110.789,32.4420,MST,2024-08-01 03:43:31.203139+00:00,2024-08-01 04:24:47.795501+00:00,2024-08-01 19:46:51.061095+00:00,2024-08-01 03:43:31.203139+00:00,2024-08-01 20:27:52.247687+00:00,2024-08-01 12:06:21+00:00,-0.0934,03:41:31.8231,03:41:31.9165,20:18:22.4631,20:18:22.5565,07:00:00.0
1,mt_lemmon,2024-08-02,-110.789,32.4420,MST,2024-08-02 03:45:16.016893+00:00,2024-08-02 04:26:18.332452+00:00,2024-08-02 19:45:11.121701+00:00,2024-08-02 03:45:16.016893+00:00,2024-08-02 20:25:58.127907+00:00,2024-08-02 12:06:17+00:00,-0.0855,03:45:28.3864,03:45:28.4719,20:22:19.0264,20:22:19.1119,07:00:00.0
2,mt_lemmon,2024-08-03,-110.789,32.4420,MST,2024-08-03 03:47:01.256892+00:00,2024-08-03 04:27:49.434727+00:00,2024-08-03 19:43:29.443046+00:00,2024-08-03 03:47:01.256892+00:00,2024-08-03 20:24:02.419120+00:00,2024-08-03 12:06:12+00:00,-0.0795,03:49:24.9478,03:49:25.0273,20:26:15.5878,20:26:15.6673,07:00:00.0
3,holmdel,2024-08-01,-74.184,40.3541,America/New_York,2024-08-01 03:43:31.203139+00:00,2024-08-01 04:24:47.795501+00:00,2024-08-01 19:46:51.061095+00:00,2024-08-01 03:43:31.203139+00:00,2024-08-01 20:27:52.247687+00:00,2024-08-01 12:06:21+00:00,-0.0945,00:41:02.2526,00:41:02.3471,19:44:18.0926,19:44:18.1871,04:00:00.0
4,holmdel,2024-08-02,-74.184,40.3541,America/New_York,2024-08-02 03:45:16.016893+00:00,2024-08-02 04:26:18.332452+00:00,2024-08-02 19:45:11.121701+00:00,2024-08-02 03:45:16.016893+00:00,2024-08-02 20:25:58.127907+00:00,2024-08-02 12:06:17+00:00,-0.0864,00:44:58.8161,00:44:58.9025,19:48:14.6561,19:48:14.7425,04:00:00.0
5,holmdel,2024-08-03,-74.184,40.3541,America/New_York,2024-08-03 03:47:01.256892+00:00,2024-08-03 04:27:49.434727+00:00,2024-08-03 19:43:29.443046+00:00,2024-08-03 03:47:01.256892+00:00,2024-08-03 20:24:02.419120+00:00,2024-08-03 12:06:12+00:00,-0.0801,00:48:55.3777,00:48:55.4579,19:52:11.2177,19:52:11.2979,04:00:00.0


In [34]:
def sidereal_time_to_degrees(hours, minutes, seconds):
    degrees = (hours * 15) + (minutes * 0.25) + (seconds * (1/240))
    return degrees

aug_1_ra = sidereal_time_to_degrees(20, 18, 22)
sep_1_ra = sidereal_time_to_degrees(22, 20, 35)

aug_1_ra, sep_1_ra

(304.59166666666664, 335.1458333333333)